This notebook preprocesses the data to have correct time and treatment metadata.

In [1]:
import pathlib
from pprint import pprint

import pandas as pd
import pyarrow.parquet as pq

In [2]:
data_subset = True
samples_per_group = 25

# path to the data
feature_selected_profiles_data_dir = pathlib.Path(
    "../../4.processing_profiled_features/data/feature_selected_data"
).resolve(strict=True)
list_of_files = list(feature_selected_profiles_data_dir.glob("*.parquet"))

input_data_dict = {
    # "first_time": {
    #     "input_file_path": list_of_files[0],
    #     "output_data_dir": pathlib.Path("../data/first_time").resolve(),
    #     "figure_dir": pathlib.Path("../figures/first_time").resolve(),
    # },
    # "within_time": {
    #     "input_file_path": list_of_files[1],
    #     "output_data_dir": pathlib.Path("../data/within_time").resolve(),
    #     "figure_dir": pathlib.Path("../figures/within_time").resolve(),
    # },
    # "pan_time": {
    #     "input_file_path": list_of_files[2],
    #     "output_data_dir": pathlib.Path("../data/pan_time").resolve(),
    #     "figure_dir": pathlib.Path("../figures/pan_time").resolve(),
    # },
    "aggregated_norm": {
        "input_file_path": pathlib.Path(
            "../data/aggregated/live_cell_pyroptosis_wave1_first_time_norm_agg.parquet"
        ).resolve(),
        "output_data_dir": pathlib.Path("../data/preprocessed").resolve(),
    },
}
pprint(input_data_dict)

{'aggregated_norm': {'input_file_path': PosixPath('/home/lippincm/Documents/pyroptosis_live-cell_timelapse/Wave1_data/4.processing_profiled_features/data/aggregated/live_cell_pyroptosis_wave1_first_time_norm_agg.parquet'),
                     'output_data_dir': PosixPath('/home/lippincm/Documents/pyroptosis_live-cell_timelapse/Wave1_data/4.processing_profiled_features/data/preprocessed')}}


In [6]:
# show all the columns
pd.set_option("display.max_columns", None)

In [7]:
df = pd.read_parquet(input_data_dict["aggregated_norm"]["input_file_path"])
print(df.shape)
df.head()

(1808, 3033)


Metadata_Well   Metadata_Plate  Cytoplasm_AreaShape_Area  \
0           C04  20241024T194653                 -0.322354   
1           C04  20241024T224849                 -0.300212   
2           C04  20241025T015040                 -0.277588   
3           C04  20241025T045229                 -0.298768   
4           C04  20241025T075429                 -0.307432   

   Cytoplasm_AreaShape_BoundingBoxArea  \
0                            -0.317583   
1                            -0.307219   
2                            -0.268199   
3                            -0.326729   
4                            -0.315754   

   Cytoplasm_AreaShape_BoundingBoxMaximum_X  \
0                                 -0.020861   
1                                 -0.009216   
2                                 -0.012799   
3                                 -0.004737   
4                                 -0.010111   

   Cytoplasm_AreaShape_BoundingBoxMaximum_Y  \
0                                 -0.023846   
1                                 -0.044054   
2                                 -0.009149   
3                                 -0.173571   
4                                 -0.083552   

   Cytoplasm_AreaShape_BoundingBoxMinimum_X  \
0                                 -0.021816   
1                                 -0.003894   
2                                 -0.004790   
3                                  0.002378   
4                                  0.005963   

   Cytoplasm_AreaShape_BoundingBoxMinimum_Y  Cytoplasm_AreaShape_Center_X  \
0                                 -0.036300                     -0.022619   
1                                 -0.035388                     -0.001874   
2                                 -0.016236                     -0.008646   
3                                 -0.162153                     -0.001047   
4                                 -0.081899                     -0.002286   

   Cytoplasm_AreaShape_Center_Y  Cytoplasm_AreaShape_Compactness  \
0                     -0.031397                        -0.016114   
1                     -0.038150                         0.015788   
2                     -0.011762                        -0.043311   
3                     -0.167759                        -0.062099   
4                     -0.077519                        -0.077738   

   Cytoplasm_AreaShape_ConvexArea  Cytoplasm_AreaShape_Eccentricity  \
0                       -0.331950                          0.187622   
1                       -0.316963                          0.170681   
2                       -0.279980                          0.172285   
3                       -0.328808                          0.121173   
4                       -0.334609                          0.160595   

   Cytoplasm_AreaShape_EquivalentDiameter  Cytoplasm_AreaShape_EulerNumber  \
0                               -0.224894                         0.113591   
1                               -0.194119                         0.113591   
2                               -0.163087                         0.113591   
3                               -0.192126                         0.113591   
4                               -0.204110                         0.113591   

   Cytoplasm_AreaShape_Extent  Cytoplasm_AreaShape_FormFactor  \
0                   -0.150742                       -0.287274   
1                   -0.189463                       -0.316365   
2                   -0.134490                       -0.261979   
3                   -0.161103                       -0.244232   
4                   -0.169396                       -0.229287   

   Cytoplasm_AreaShape_MajorAxisLength  Cytoplasm_AreaShape_MaxFeretDiameter  \
0                            -0.273354                             -0.266416   
1                            -0.242261                             -0.232689   
2                            -0.216064                             -0.225780   
3                            -0.267060          

In [5]:
for dataset in input_data_dict:
    input_data_dict[dataset]["output_data_dir"].mkdir(parents=True, exist_ok=True)
    if data_subset:
        subset_data_output_file_path = pathlib.Path(
            input_data_dict[dataset]["output_data_dir"]
            / f'{input_data_dict[dataset]["input_file_path"].stem}_subset_testing_data.parquet'
        ).resolve()
        data = pd.read_parquet(
            input_data_dict[dataset]["input_file_path"], columns=["Metadata_Well"]
        )
        data_df = data.groupby("Metadata_Well").apply(
            lambda x: x.sample(min(len(x), samples_per_group), random_state=0)
        )

        # get the indexes of the data
        data_idx = data.index
        data = pd.concat(
            [
                pd.read_parquet(
                    input_data_dict[dataset]["input_file_path"], columns=[col]
                ).iloc[data_idx]
                for col in pq.read_schema(
                    input_data_dict[dataset]["input_file_path"]
                ).names
            ],
            axis=1,
        )
        # save the subset data
        data.to_parquet(subset_data_output_file_path)
        data.head()
    else:
        data = pd.read_parquet(input_data_dict[dataset]["input_file_path"])
        data.head()

    # perform preprocessing on each data
    # sort the time and replace with 1, 2, 3, 4
    time_mapping = {
        time: i for i, time in enumerate(data["Metadata_Plate"].sort_values().unique())
    }
    # check if the new columns exist, if so drop them
    if "Metadata_treatment_serum" in data.columns:
        data.drop(columns=["Metadata_treatment_serum"], inplace=True)
    if "Metadata_Time" in data.columns:
        data.drop(columns=["Metadata_Time"], inplace=True)
    # Combine all new columns at once to avoid fragmentation
    new_columns = pd.DataFrame(
        {
            "Metadata_treatment_serum": data["Metadata_treatment"]
            + " "
            + data["Metadata_serum"],
            "Metadata_Time": data["Metadata_Plate"].map(time_mapping),
        }
    )
    data = pd.concat([data, new_columns], axis=1)

    # drop Wells N04, N06, N08, and N10 as they have no Hoechst stain

    data = data[~data["Metadata_Well"].str.contains("N04|N06|N08|N10")]

    if data_subset:
        data.to_parquet(subset_data_output_file_path)
    else:
        # over write the current parquet file
        data.to_parquet(input_data_dict[dataset]["input_file_path"])

    print(f"Preprocessed data for {dataset} has the shape: {data.shape}")

/tmp/ipykernel_329796/2831818137.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_df = data.groupby("Metadata_Well").apply(


KeyError: 'Metadata_treatment'

In [ ]:
data.head()